In [1]:
import torch
print("CUDA beschikbaar:", torch.cuda.is_available())
print("Aantal GPU's:", torch.cuda.device_count())
print(torch.version.cuda)  # Shows the CUDA version PyTorch was built with



CUDA beschikbaar: True
Aantal GPU's: 1
12.1


# Object detection model training

In [7]:
from ultralytics import YOLO
import os

# === dataset root ===
DATASET_DIR = r"C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final"

# set the paths
train_images = os.path.join(DATASET_DIR, "train", "images").replace("\\", "/")
val_images = os.path.join(DATASET_DIR, "val", "images").replace("\\", "/")
test_images = os.path.join(DATASET_DIR, "test", "images").replace("\\", "/")

# YAML folder create
yaml_path = os.path.join(DATASET_DIR, "data.yaml")

yaml_content = f"""
train: "{train_images}"
val: "{val_images}"
test: "{test_images}"

nc: 10
names:
  0: "3031"
  1: "6714"
  2: "5953"
  3: "5121"
  4: "2351"
  5: "2991"
  6: "2392"
  7: "6315"
  8: "10896"
  9: "7243"
"""

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)

print("✅ YAML has been written:")
print(yaml_path)


✅ YAML has been written:
C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final\data.yaml


### 2.step rematching the labels

In [8]:
import os
from pathlib import Path
from shutil import copy2

DATASET_DIR = Path(r"C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final")

id_map = {
    3031: 0,
    6714: 1,
    5953: 2,
    5121: 3,
    2351: 4,
    2991: 5,
    2392: 6,
    6315: 7,
    10896: 8,
    7243: 9,
}

def remap_labels(split):
    labels_dir = DATASET_DIR / split / "labels"
    backup_dir = DATASET_DIR / f"_backup_labels_{split}"
    backup_dir.mkdir(exist_ok=True, parents=True)

    changed, skipped = 0, 0
    for txt in labels_dir.glob("*.txt"):
        # backup
        copy2(txt, backup_dir / txt.name)

        new_lines = []
        modified = False
        with open(txt, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                try:
                    cls = int(float(parts[0]))
                except ValueError:
                    new_lines.append(line)
                    continue

                if cls in id_map:
                    new_cls = id_map[cls]
                    if new_cls != cls:
                        parts[0] = str(new_cls)
                        modified = True
                    new_lines.append(" ".join(parts) + "\n")
                elif 0 <= cls <= 9:
                    new_lines.append(line if line.endswith("\n") else line + "\n")
                else:
                    skipped += 1
                
        if modified:
            with open(txt, "w", encoding="utf-8") as f:
                f.writelines(new_lines)
            changed += 1
    return changed, skipped

total_changed, total_skipped = 0, 0
for split in ["train", "val", "test"]:
    ch, sk = remap_labels(split)
    total_changed += ch
    total_skipped += sk
    print(f"{split}: changed folder={ch}, skipped/warning raw={sk}")

print("Done. backup folders: _backup_labels_train/val/test")


train: changed folder=0, skipped/warning raw=0
val: changed folder=0, skipped/warning raw=0
test: changed folder=0, skipped/warning raw=0
Done. backup folders: _backup_labels_train/val/test


### 3. step quick data check

In [9]:
from pathlib import Path

for split in ["train", "val", "test"]:
    img_dir = DATASET_DIR / split / "images"
    lbl_dir = DATASET_DIR / split / "labels"
    imgs = {p.stem for p in img_dir.glob("*.*")}
    lbls = {p.stem for p in lbl_dir.glob("*.txt")}
    missing_lbl = imgs - lbls
    missing_img = lbls - imgs
    print(f"[{split}] images={len(imgs)} labels={len(lbls)} | labels loss={len(missing_lbl)} | images loss (only label, no image)={len(missing_img)}")
    if missing_lbl:
        print("  no label samples:", list(sorted(missing_lbl))[:5])
    if missing_img:
        print("  no image samples:", list(sorted(missing_img))[:5])


[train] images=498 labels=498 | labels loss=0 | images loss (only label, no image)=0
[val] images=174 labels=174 | labels loss=0 | images loss (only label, no image)=0
[test] images=94 labels=94 | labels loss=0 | images loss (only label, no image)=0


### 4 train yolo

In [1]:
from ultralytics import YOLO
import torch

print("CUDA:", torch.cuda.is_available(), " GPUs:", torch.cuda.device_count())
print("torch cuda built:", torch.version.cuda)

DATASET_DIR = r"C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final"
yaml_path = f"{DATASET_DIR}/data.yaml"

model = YOLO("yolov8m.pt")

results = model.train(
    data=yaml_path,
    epochs=60,
    imgsz=640,
    batch=8,
    patience=10,
    optimizer="AdamW",
    project="insect_yolo_results",
    device=0
)

metrics = model.val(split="test")
pred_dir = f"{DATASET_DIR}/test/images"
model.predict(source=pred_dir, save=True, conf=0.25)


CUDA: True  GPUs: 1
torch cuda built: 12.1
New https://pypi.org/project/ultralytics/8.3.222 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.207  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '3031', 1: '6714', 2: '5953', 3: '5121', 4: '2351', 5: '2991', 6: '2392', 7: '6315', 8: '10896', 9: '7243'}
 obb: None
 orig_img: array([[[148, 113, 133],
         [183, 148, 168],
         [190, 155, 175],
         ...,
         [110,  85, 105],
         [118,  93, 113],
         [148, 123, 143]],
 
        [[145, 110, 130],
         [172, 137, 157],
         [183, 148, 168],
         ...,
         [118,  93, 113],
         [118,  93, 113],
         [140, 115, 135]],
 
        [[143, 108, 128],
         [161, 126, 146],
         [181, 147, 164],
         ...,
         [142, 120, 139],
         [141, 116, 136],
         [153, 128, 148]],
 
        ...,
 
        [[162, 136, 150],
         [159, 133, 147],
         [176, 150, 164],
         ...,
         [157, 144, 152],
         [154, 141, 149],
         [152, 139, 147]],
 
        [[14

## Load the trained model (best.pt saved automatically after training)

In [3]:
model = YOLO("./insect_yolo_results/train8/weights/best.pt")

## Evaluate on your test set

In [4]:
metrics = model.val(data=r"C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final/data.yaml", split="test")

print("mAP50:", metrics.box.map50)
print("mAP50-95:", metrics.box.map)
print("Precision:", metrics.box.p)
print("Recall:", metrics.box.r)

Ultralytics 8.3.207  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 92 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 921.2268.7 MB/s, size: 1326.5 KB)
val: Scanning C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\labels.cache... 94 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 94/94  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.7it/s 8.5s0.5s
                   all         94        264      0.331      0.519      0.453      0.288
                  3031         25         32      0.324      0.688      0.437      0.201
                  6714         26         33       0.32      0.758      0.571      0.437
                  5953         17         37      0.322      0.833      0.782      0.589
                  5121         35      

Run inference on test images and save visualized predictions,

In [5]:
model.predict(
    source=r"C:/Users/ofaru/Desktop/AI for Society/Group Project/Project Itself/3. branch/insects/dataset_yolo_final/test/images",
    save=True,
    conf=0.25
)


image 1/94 C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\images\11-20220901220000-snapshot.jpg: 352x640 2 3031s, 12.6ms
image 2/94 C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\images\120-20230124223759-snapshot.jpg: 352x640 (no detections), 14.8ms
image 3/94 C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\images\166-20230124020000-snapshot.jpg: 352x640 1 6714, 1 5953, 1 2991, 12.1ms
image 4/94 C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\images\20-20220716220000-snapshot.jpg: 352x640 1 3031, 2 5953s, 12.8ms
image 5/94 C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\images\20220522001422-00-50.jpg: 352x640 2 2351s, 12.7ms
image 6/94 C:\Users\ofaru\Desktop\AI for Society\Group Pr

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '3031', 1: '6714', 2: '5953', 3: '5121', 4: '2351', 5: '2991', 6: '2392', 7: '6315', 8: '10896', 9: '7243'}
 obb: None
 orig_img: array([[[148, 113, 133],
         [183, 148, 168],
         [190, 155, 175],
         ...,
         [110,  85, 105],
         [118,  93, 113],
         [148, 123, 143]],
 
        [[145, 110, 130],
         [172, 137, 157],
         [183, 148, 168],
         ...,
         [118,  93, 113],
         [118,  93, 113],
         [140, 115, 135]],
 
        [[143, 108, 128],
         [161, 126, 146],
         [181, 147, 164],
         ...,
         [142, 120, 139],
         [141, 116, 136],
         [153, 128, 148]],
 
        ...,
 
        [[162, 136, 150],
         [159, 133, 147],
         [176, 150, 164],
         ...,
         [157, 144, 152],
         [154, 141, 149],
         [152, 139, 147]],
 
        [[14

In [6]:
metrics = model.val(data=yaml_path, split="test", project="insect_yolo_results", name="test_metrics", device=0)

Ultralytics 8.3.207  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 1359.6139.5 MB/s, size: 1468.4 KB)
val: Scanning C:\Users\ofaru\Desktop\AI for Society\Group Project\Project Itself\3. branch\insects\dataset_yolo_final\test\labels.cache... 94 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 94/94 93.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.7it/s 8.7s0.5s
                   all         94        264      0.331      0.519      0.453      0.288
                  3031         25         32      0.324      0.688      0.437      0.201
                  6714         26         33       0.32      0.758      0.571      0.437
                  5953         17         37      0.322      0.833      0.782      0.589
                  5121         35         46      0.174      0.174      0.193      0.149
                  2351